In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Luminesce Syntax

Attributes
----------

"""

toggle_code("Toggle Docstring")


*Setting up the environment for the tutorial*

This section of the notebook creates a portfolio, some equities, and a file in drive to run luminesce queries against.

In [2]:
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid as lu
import pandas as pd
import lusid_drive
import lusid_drive.rest
import lusid.rest
from lusid_drive.rest import ApiException
from pprint import pprint
import json


secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10288.0,0.5.3083,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [3]:
# here's some sample equities we've prepared earlier
ftse_instruments = pd.read_csv('_data/uk_equity_instruments.csv')
nasdaq_instruments = pd.read_csv('_data/us_equity_instruments.csv')

In [4]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

# create portfolio for this module
def upsert_portfolio(api_factory, scope, code, portfolio_name, portfolio_description):
    api_instance = api_factory.build(lu.TransactionPortfoliosApi)
        # Create an instance of the API class
    create_transaction_portfolio_request = {"displayName":portfolio_name,"description":portfolio_description,"code":code,"baseCurrency":"GBP", "instrumentScopes":["FBNUniversityModule-T2-2"]} # CreateTransactionPortfolioRequest | The definition of the transaction portfolio.
    try:
        # UpdatePortfolio: Update portfolio
        api_response = api_instance.create_portfolio(scope, create_transaction_portfolio_request)
        pprint(api_response)
    except lusid.rest.ApiException as e:
        if not exception_guard(e, 112):
            print("Exception when calling PortfoliosApi->update_portfolio: %s\n" % e)

scope = 'FBNUniversity' # str | The scope of the portfolio.
code = 'Module-T2-2' # str | The code of the portfolio. Together with the scope this uniquely identifies the portfolio.
portfolio_name = "Training module T2.2"
portfolio_description = "A portfolio to hold transactions"
upsert_portfolio(api_factory, scope, code, portfolio_name, portfolio_description)

In [5]:
# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier):
    request_id = name + isin
    request ={"name":name,
                     "identifiers":{ "Figi":{"value":figi},
                                     "Isin":{"value":isin},
                                     other_identifier_name:{"value":other_identifier}
                                   },

                     "definition":{"instrumentType":"Equity", "domCcy":ccy}}
    return request_id, request

In [6]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)

# We're gonna load some american and UK equities into LUSID

other_identifier_name = 'Sedol'
ccy = 'GBP'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in ftse_instruments.itertuples())
ftse_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}
other_identifier_name = 'Cusip'
ccy = 'USD'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in nasdaq_instruments.itertuples())
nasdaq_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}

request_body_dict = nasdaq_body_dict | ftse_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
#     pprint(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)


In [7]:

# let's chuck a file containing a randomly selected subset of 25 instruments with their sectors into drive for some examples:

subset_file = open('_data/instrument_subset.csv').read()

drive_config = lusid_drive.Configuration(
    host = f'{api_instance.api_client.configuration.host[:-4]}/drive'
)
drive_config.access_token = RefreshingToken()

# Enter a context with an instance of the API client
with lusid_drive.ApiClient(drive_config) as api_client:
    # Create an instance of the API class
    api_instance = lusid_drive.FoldersApi(api_client)
    create_folder = {"path":"/","name":"finbourne university"} # CreateFolder | A CreateFolder object that defines the name and path of the new folder

    try:
        # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
        api_response = api_instance.create_folder(create_folder)
        pprint(api_response)
    except ApiException as e:
        if not exception_guard(e, 664):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        
        
    api_instance = lusid_drive.FilesApi(api_client)
    x_lusid_drive_filename = 'ftse_subset.csv' # str | File name.
    x_lusid_drive_path = '/finbourne university/' # str | File path.
    content_length = len(subset_file.encode('UTF-8'))
    body = subset_file # str | 

    try:
        # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
        api_response = api_instance.create_file(x_lusid_drive_filename, x_lusid_drive_path, content_length, body)
    except lusid_drive.rest.ApiException as e:
        if not exception_guard(e, 671):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        

# Luminesce Syntax

In this part of the course we will provide some additional technical information about luminesce, and demonstrate how luminesce queries can be used to extract information from your data.

## What is Structured Query Language (SQL)


SQL - pronounced "sequel" was first developed at IBM in the early 1970s, as a way to query and manipulate structured relational databases. SQL is a set-based, declarative programming language. Many implementations of SQL add extensions with procedural programming functionality, such as control-of-flow contructs.

## Luminesce Queries - SQL with extensions


SQLite is the most widely deployed database engine, used by several of the top web browsers, operating systems and application software.

Luminesce supports most of the query syntax for the SQLite dialect of SQL, but not the data manipulation or data definition syntax. Data can still be manipulated using some extensions to the language that Luminesce adds. 

Luminesce adds extensions to the SQLite implementation which can:
- exclude columns from queries
- automatically select important columns in queries
- parameterise queries
- write data to some Luminesce providers
- wait on results of some dependent operation, or for a time period
- run queries in parallel on an iterated range of parameters.



## Luminesce providers


A provider is a component that enables you to write a Luminesce SQL query for a data source in situ, without first having to extract, transform or load data from that source.

We supply providers for numerous data sources, including for the investment data stored in LUSID itself, for files stored in Drive and for some external data providers. You can also create your own provider for a proprietary data source.

There are two types of provider: a data provider and a direct provider.

A data provider is designed to query a data source whose shape is known. It therefore returns a table of results with a fixed number of fields (columns).

A direct provider is designed to query a data source whose shape is not known, and thus cannot return a table of results with a fixed number of fields (columns).

We'll use both data and direct providers in this tutorial.

## Use case: Using Luminesce to query and join data from disparate sources, and to distribute the resulting information.


I'd like to read a file from Drive and use the data from this file to filter transaction results. I'd like to join these filtered results with another provider and distribute the results. This will be packaged up into a simple to use View which runs on a schedule.

## An intro to SQLite queries

### A query using a simple SELECT statement


Its simple to get data from a provider using a select statement.

Use the following statement to get all data from a provider:

`SELECT * FROM SOME.PROVIDER`

Or retrieve data from a subset of columns by specifying them:

`SELECT A_COLUMN, ANOTHER_COLUMN FROM SOME.PROVIDER`

Here we'll use a SELECT statement to query for all of our porfolios.

In [8]:
%%luminesce
SELECT * 
FROM Lusid.Portfolio
LIMIT 100

,PortfolioScope,PortfolioCode,PortfolioType,BaseCurrency,DisplayName,Description,OriginPortfolioScope,OriginPortfolioCode,ParentPortfolioScope,ParentPortfolioCode,ChildPortfolioIds,CorporateActionSourceScope,CorporateActionSourceId,AccountingMethod,SubHoldingKeys,Created,InstrumentScopes
0,FBNUniversity,Module-T2-2,Transaction,GBP,Training module T2.2,A portfolio to hold transactions,FBNUniversity,Module-T2-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-11-03 20:14:15.334,FBNUniversityModule-T2-2
1,FBNUniversity,Module-4-1,Transaction,GBP,Training module 4.1,NaN,FBNUniversity,Module-4-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-4-1
2,FBNUniversity,Module-3-1,Transaction,GBP,Training module 3.1,NaN,FBNUniversity,Module-3-1,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/Module-3-1,2022-01-01,FBNUniversityModule-3-1
3,FBNUniversity,Module-6-2,Transaction,GBP,"Training module 6.2 - Same transaction code, d...",NaN,FBNUniversity,Module-6-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-2
4,FBNUniversity,Module-6-4,Transaction,GBP,Training module 6.4 - Track commission as sepa...,NaN,FBNUniversity,Module-6-4,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/TrackCommission,2022-01-01,FBNUniversityModule-6-4
5,FBNUniversity,Module-6-3,Transaction,GBP,Training module 6.3 - Reduce cash balance by c...,NaN,FBNUniversity,Module-6-3,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-3
6,FBNUniversity,Module-3-2,Transaction,GBP,Training module 3.2,NaN,FBNUniversity,Module-3-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-3-2
7,FBNUniversity,Module-1,Transaction,GBP,Module 1,NaN,FBNUniversity,Module-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2021-12-01,FBNUniversityModule-1
8,FBNUniversity,Module-6-1,Transaction,GBP,Training module 6.1 - Different transaction co...,NaN,FBNUniversity,Module-6-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-1


In this example, we've selected all columns and rows from the Lusid.Portfolio provider. We can specify column names if we are only interested in some parts of the data.

In [9]:
%%luminesce
SELECT DisplayName, PortfolioCode 
FROM Lusid.Portfolio
LIMIT 100

,DisplayName,PortfolioCode
0,Training module 6.4 - Track commission as sepa...,Module-6-4
1,Training module 3.2,Module-3-2
2,"Training module 6.2 - Same transaction code, d...",Module-6-2
3,Training module 6.3 - Reduce cash balance by c...,Module-6-3
4,Training module T2.2,Module-T2-2
5,Training module 3.1,Module-3-1
6,Training module 6.1 - Different transaction co...,Module-6-1
7,Training module 4.1,Module-4-1
8,Module 1,Module-1


Here I really just want to see the name of the portfolio and the portfolio code, I've selected only those columns from the provider.

### Limiting the number of return results using LIMIT


These simple SELECT statements are useful for getting an idea of the shape and quality of the data returned by a provider, but they tend to return a lot of data. Lets use LIMIT to reduce the amount of data requested:

In [10]:
%%luminesce
SELECT DisplayName, PortfolioCode 
FROM Lusid.Portfolio 
LIMIT 5

,DisplayName,PortfolioCode
0,Training module T2.2,Module-T2-2
1,Training module 4.1,Module-4-1
2,Training module 3.1,Module-3-1
3,"Training module 6.2 - Same transaction code, d...",Module-6-2
4,Training module 6.4 - Track commission as sepa...,Module-6-4


This example returns the first 5 rows of data from the Lusid.Portfolio provider.

### Filter queries WHERE ...


We can now filter these results down - lets get all of the Equities that are listed in GBP


In [11]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity 
WHERE DomCcy = 'GBP'
LIMIT 100

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003D8A,GB00BLGZ9862,BLGZ986,NaN,NaN,NaN,BBG000BF46Y8,NaN,NaN,NaN,...,fbn-uni-T02002,Tesco,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
1,LUID_00003D6L,GB00B0744B38,B0744B3,NaN,NaN,NaN,BBG000BD7MX6,NaN,NaN,NaN,...,default,Bunzl,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DD5,GB0002634946,263494,NaN,NaN,NaN,BBG000BD5TW4,NaN,NaN,NaN,...,FBNUniversity,BAE Systems,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003DC7,GB00BDR05C01,BDR05C0,NaN,NaN,NaN,BBG000FV67Q4,NaN,NaN,NaN,...,fbn-uni-T02002,National Grid plc,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003DAB,GB00B0744B38,B0744B3,NaN,NaN,NaN,BBG000BD7MX6,NaN,NaN,NaN,...,fbn-uni-T02002,Bunzl,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_00003D53,GB00B1KJJ408,NaN,NaN,WTB LN,imd_35349900,BBG000BRVH05,NaN,NaN,NaN,...,FBNUniversityModule-1,Whitbread,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
96,LUID_00003DBZ,GB0033195214,3319521,NaN,NaN,NaN,BBG000BKH1W6,NaN,NaN,NaN,...,fbn-uni-T02002,Kingfisher,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
97,LUID_00003DF0,JE00BJVNSS43,BJVNSS4,NaN,NaN,NaN,BBG00NZJ0JG0,NaN,NaN,NaN,...,FBNUniversity,Ferguson plc,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
98,LUID_00003DDA,GB0002162385,216238,NaN,NaN,NaN,BBG000DDCSJ1,NaN,NaN,NaN,...,FBNUniversity,Aviva,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


We've filtered our query for equities who have a domestic currency of Pound Sterling.

### JOINing disparate datasources with join


Now, I've got a file in drive with sector information for some of these instruments. I'm going to join data from the file using our Drive provider with the saved instrument data so I can see the sectors for instruments where we know this information.

First lets get our instrument data.

Then we'll grab the file contents from drive.

We'll take a look at cross-joins, which SQLite uses to build left joins and inner joins. Cross-joins return the cartesian product of both tables - each row on the left table is concatenated with every row on the right table, to form a massive table with every combination of both tables' rows:


In [12]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector 
FROM Lusid.Instrument.Equity as Equities 
CROSS JOIN @x as File
LIMIT 100

,DisplayName,DomCcy,Sector
0,Diageo,GBP,Financial Services
1,Diageo,GBP,General Industrials
2,Diageo,GBP,Travel & Leisure
3,Diageo,GBP,Equity Investment Instruments
4,Diageo,GBP,Household Goods & Home Construction
...,...,...,...
95,Aviva,GBP,Mining
96,Aviva,GBP,Media
97,Aviva,GBP,Mobile Telecommunications
98,Aviva,GBP,Food & Drug Retailers


We've returned a table with every combination of Lusid Equities with every row in our file.

In a left join, all rows from the left table are returned, along with rows in the right table that match the supplied key constraint. If there is a row in the left table with no matching row on the right, the columns that come from the right table in the resulting row are set to null.

In [13]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM Lusid.Instrument.Equity as Equities 
LEFT JOIN @x as File on Equities.Figi = File.Figi
LIMIT 100

,DisplayName,DomCcy,Sector,Figi
0,Tesco,GBP,NaN,BBG000BF46Y8
1,Bunzl,GBP,NaN,BBG000BD7MX6
2,"Okta, Inc.",USD,NaN,BBG001YV1SM4
3,BAE Systems,GBP,NaN,BBG000BD5TW4
4,Intel Corporation,USD,NaN,BBG000C0G1D1
...,...,...,...,...
95,Halma,GBP,NaN,BBG000BDLT47
96,Standard Life Aberdeen,GBP,NaN,BBG000N2C023
97,Smith & Nephew,GBP,NaN,BBG000BF2206
98,Cintas Corporation,USD,NaN,BBG000D2RM57


We see this here as there are some companies with no matching sector in our file - these are returned as null.

We are going to ignore instruments without a matching sector for now, so we'll use an inner join.
Inner joins only return rows where the joining key has a match on both sides of the table:

In [14]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM Lusid.Instrument.Equity as Equities 
INNER JOIN @x as File on Equities.Figi = File.Figi
LIMIT 100

,DisplayName,DomCcy,Sector,Figi
0,Aviva,GBP,Life Insurance,BBG000DDCSJ1
1,BP,GBP,Oil & Gas Producers,BBG000C05BD1
2,Ocado Group,GBP,Food & Drug Retailers,BBG000BDXZH6
3,Land Securities,GBP,Real Estate Investment Trusts,BBG000BDPHS3
4,BP,GBP,Oil & Gas Producers,BBG000C05BD1
...,...,...,...,...
65,Taylor Wimpey,GBP,Household Goods & Home Construction,BBG000BF4KL1
66,Rightmove,GBP,Media,BBG000DD5X08
67,Taylor Wimpey,GBP,Household Goods & Home Construction,BBG000BF4KL1
68,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7


Now we have a joined table where only Equities with a sector on file are returned, along with their sector.

### GROUPing results BY some condition in query


We can also group our results by a column - we'll perform an aggregation and group-by to count the number of instrument in each sector.

There are many different aggregation functions in SQLite, such as sum, avg and count.

Sum will return the sum of the specified field in each group.
Average returns the mean of the specified field in each group.
Count returns the number of results that are non-null for the field in each group. We can also specify that count returns the count of distinct values [Count(Distinct <field>)], or all rows [count(*)], in each group.

In [15]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT count(File.Sector), File.Sector
FROM Lusid.Instrument.Equity as Equities 
INNER JOIN @x as File on Equities.Figi = File.Figi
GROUP BY File.Sector
LIMIT 100

,count(File_Sector COLLATE BINARY),Sector
0,2,Automobiles & Parts
1,2,Equity Investment Instruments
2,4,Financial Services
3,2,Food & Drug Retailers
4,2,General Industrials
5,3,Household Goods & Home Construction
6,2,Investment Services
7,9,Life Insurance
8,2,Media
9,7,Mining


We've now grouped our results by sector, and for each sector, we''ve returned how many Equities belong to the sector.

## Introducing some Luminesce extensions


### Using ^ to SELECT the most important columns from a provider


Let's use the carat symbol to grab then more important columns in the Equity provider. The list of important columns is decided by the writer of the Provider.

In [16]:
%%luminesce
SELECT ^ 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive
0,LUID_00003D6S,default,Diageo,Equity,Equities,GBP,True
1,LUID_00003DH7,FBNUniversity,Sainsbury's,Equity,Equities,GBP,True
2,LUID_00003DH2,FBNUniversity,Kingfisher,Equity,Equities,GBP,True
3,LUID_00003D6A,default,Aviva,Equity,Equities,GBP,True
4,LUID_00003DDT,FBNUniversity,Incyte Corporation,Equity,Equities,USD,True
...,...,...,...,...,...,...,...
95,LUID_00003DG6,FBNUniversity,Croda International,Equity,Equities,GBP,True
96,LUID_00003D6Q,default,Croda International,Equity,Equities,GBP,True
97,LUID_00003DCW,FBNUniversity,Reckitt Benckiser,Equity,Equities,GBP,True
98,LUID_00003D96,fbn-uni-T02002,Schroders,Equity,Equities,GBP,True


In the Lusid Equities provider: LusidInstrumentId, Scope, DisplayName, InferredAssetClass, InferredDomCcy and IsActive are the fields marked by the provider as being most important.

### Excluding columns from the results of a query using the except keyword.


Using some providers, we can also exclude columns from the result of a query by using the except keyword:

In [17]:
%%luminesce
SELECT ^ (except Type, IsActive) 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,Scope,DisplayName,InferredAssetClass,InferredDomCcy
0,LUID_00003D8A,fbn-uni-T02002,Tesco,Equities,GBP
1,LUID_00003D6L,default,Bunzl,Equities,GBP
2,LUID_00003DDV,FBNUniversity,"Okta, Inc.",Equities,USD
3,LUID_00003DD5,FBNUniversity,BAE Systems,Equities,GBP
4,LUID_00003DAK,fbn-uni-T02002,Intel Corporation,Equities,USD
...,...,...,...,...,...
95,LUID_00003D4W,FBNUniversityModule-1,Halma,Equities,GBP
96,LUID_00003DA3,fbn-uni-T02002,Standard Life Aberdeen,Equities,GBP
97,LUID_00003D89,fbn-uni-T02002,Smith & Nephew,Equities,GBP
98,LUID_00003DEH,FBNUniversity,Cintas Corporation,Equities,USD


Here we select the most important columns again, but exclude the Type and IsActive columns

### Using parameters to filter queries


Many providers will have parameters that can also be used to filter a Luminesce query as part of a standard WHERE clause.

We can see these parameters by querying the Sys.Field Provider.

In [18]:
%%luminesce
SELECT FieldName, DataType, ParamDefaultValue, Description 
FROM Sys.Field 
WHERE TableName = 'Lusid.Instrument.Equity' 
AND FieldType = 'Parameter'
LIMIT 100

,FieldName,DataType,ParamDefaultValue,Description
0,AsAt,DateTime,0001-01-01T00:00:00Z,As at time to query at. Defaults to latest.
1,EffectiveAt,DateTime,0001-01-01T00:00:00Z,Effective time to query at. Defaults to latest.
2,UseLusidFilter,Boolean,True,Should the filter be translated to a Finbourne...


We can see there are 3 fields we can filter our query on: AsAt, EffectiveAt and UseLusidFilter

In [19]:
%%luminesce

SELECT * 
FROM Lusid.Instrument.Equity
WHERE UseLusidFilter = False
LIMIT 100

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003D6S,GB0002374006,237400,NaN,NaN,NaN,BBG000BS69D5,NaN,NaN,NaN,...,default,Diageo,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
1,LUID_00003DH7,GB00B019KW72,B019KW7,NaN,NaN,NaN,BBG000BF0KW3,NaN,NaN,NaN,...,FBNUniversity,Sainsbury's,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DH2,GB0033195214,3319521,NaN,NaN,NaN,BBG000BKH1W6,NaN,NaN,NaN,...,FBNUniversity,Kingfisher,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003D6A,GB0002162385,216238,NaN,NaN,NaN,BBG000DDCSJ1,NaN,NaN,NaN,...,default,Aviva,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003DDT,US45337C1027,NaN,45337C102,NaN,NaN,BBG000BNPSQ9,NaN,NaN,NaN,...,FBNUniversity,Incyte Corporation,NaN,NaN,Equity,Equities,USD,True,NaN,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_00003DG6,GB00BJFFLV09,BJFFLV0,NaN,NaN,NaN,BBG000BDCLS8,NaN,NaN,NaN,...,FBNUniversity,Croda International,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
96,LUID_00003D6Q,GB00BJFFLV09,BJFFLV0,NaN,NaN,NaN,BBG000BDCLS8,NaN,NaN,NaN,...,default,Croda International,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
97,LUID_00003DCW,GB00B24CGK77,B24CGK7,NaN,NaN,NaN,BBG000BX8BP4,NaN,NaN,NaN,...,FBNUniversity,Reckitt Benckiser,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
98,LUID_00003D96,GB0002405495,240549,NaN,NaN,NaN,BBG000BF0TF3,NaN,NaN,NaN,...,fbn-uni-T02002,Schroders,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


Here we've filtered an Equity query on the UseLusidFilter. Notice there is no returned Field called UseLusidFilter - its not a column in our data.

## Using Luminesce variables


You can use variables to create and populate arbitrary tables of data that can then be used as part of a Luminesce query for either a data provider or a direct provider.

Declare a variable that represents a table of data using:
`@variable`

Or to represent a single value:
`@@variable`

Let's see variables in action. 

In [20]:
%%luminesce
@instruments_in_GBP = SELECT * 
                      FROM Lusid.Instrument.Equity 
                      WHERE DomCcy = 'GBP';
SELECT Isin 
FROM @instruments_in_GBP
LIMIT 100

,Isin
0,GB00BLGZ9862
1,GB00B0744B38
2,GB0002634946
3,GB00BDR05C01
4,GB00B0744B38
...,...
93,GB00B1KJJ408
94,GB0033195214
95,JE00BJVNSS43
96,GB0002162385


Here_instruments_in GBP is a table variable, set from the results of the query on equities.

In [21]:
%%luminesce
@sectors = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

@@selected_sector = SELECT 'Life Insurance';
SELECT * 
FROM @sectors 
WHERE Sector = @@selected_sector
LIMIT 100

,Name,Sector,Figi
0,Prudential plc,Life Insurance,BBG000BDY322
1,St. James's Place plc,Life Insurance,BBG000BRMPC4
2,Legal & General,Life Insurance,BBG000BDQCG6
3,Aviva,Life Insurance,BBG000DDCSJ1


I've set selected_sector as a single-value variable "Lise Insurance", and filtered my file by sector.

## Introducing more advanced Luminesce extensions


You can use the CROSS APPLY or OUTER APPLY statement to iterate over a set of parameters. This is useful to execute a Luminesce query in parallel for each of the parameter inputs.

### Cross-apply

Cross-apply is similar to INNER JOIN, in that only records that have matching values on both sides are returned. 
We'll run some equity queries in parallel using Tools.split to create a set of currencies to filter by- the table will contain the currency joined to all of the returned equity tables:

In [22]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';

SELECT c.val as selectedCurrency, results.*
FROM @currencies c
CROSS APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity equity
       WHERE equity.DomCcy = c.val
       
) results
LIMIT 100

,selectedCurrency,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D6S,default,Diageo,Equity,Equities,GBP,True,1
1,GBP,LUID_00003DH7,FBNUniversity,Sainsbury's,Equity,Equities,GBP,True,1
2,GBP,LUID_00003DH2,FBNUniversity,Kingfisher,Equity,Equities,GBP,True,1
3,GBP,LUID_00003D6A,default,Aviva,Equity,Equities,GBP,True,1
4,GBP,LUID_00003D56,FBNUniversityModule-1,BP,Equity,Equities,GBP,True,1
...,...,...,...,...,...,...,...,...,...
95,GBP,LUID_00003D9B,fbn-uni-T02002,British American Tobacco,Equity,Equities,GBP,True,1
96,GBP,LUID_00003D55,FBNUniversityModule-1,Glencore,Equity,Equities,GBP,True,1
97,GBP,LUID_00003DBI,fbn-uni-T02002,Rio Tinto,Equity,Equities,GBP,True,1
98,GBP,LUID_00003DDM,FBNUniversity,Severn Trent,Equity,Equities,GBP,True,1


This example works similar to an inner join, we've run a query in parallel for some currency values, returning a similar table to an inner join. Remember, inner join returns only rows which fulfil the join criteria.

### Outer-apply


Outer apply is similar to LEFT OUTER JOIN, in that all records on the left side are returned, even when there are no matching records on the right side.

In [23]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';
SELECT c.val, results.*
FROM @currencies c
OUTER APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity as equity
       WHERE equity.DomCcy = c.val
) results
LIMIT 100

,val,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D4V,FBNUniversityModule-1,RELX,Equity,Equities,GBP,True,1
1,GBP,LUID_00003D4W,FBNUniversityModule-1,Halma,Equity,Equities,GBP,True,1
2,GBP,LUID_00003D4X,FBNUniversityModule-1,Unilever,Equity,Equities,GBP,True,1
3,GBP,LUID_00003D4Y,FBNUniversityModule-1,Just Eat Takeawy,Equity,Equities,GBP,True,1
4,GBP,LUID_00003D4Z,FBNUniversityModule-1,Mondi,Equity,Equities,GBP,True,1
...,...,...,...,...,...,...,...,...,...
95,GBP,LUID_00003D8S,fbn-uni-T02002,Fresnillo,Equity,Equities,GBP,True,1
96,GBP,LUID_00003D8W,fbn-uni-T02002,Royal Dutch Shell,Equity,Equities,GBP,True,1
97,GBP,LUID_00003D8X,fbn-uni-T02002,Compass Group,Equity,Equities,GBP,True,1
98,GBP,LUID_00003D8Y,fbn-uni-T02002,Berkeley Group Holdings,Equity,Equities,GBP,True,1


We've run a query in parallel for some currency values, returning a similar table to a left join with our selected currencies on the left side of the join. Here, the values from Lusid.Instrument.Equity on the CHF row are null.